# Simulação da Transmissão de uma Onda Dente-de-Serra por Canais Passa-Baixas

O objetivo deste notebook é simular o que ocorre quando tentamos transmitir um certo sinal por canais com características de um passa-baixas. No nosso exemplo, vamos considerar, para fins de ilustração, um sinal emitido com a forma de uma onda dente-de-serra. A partir daí, vamos tentar entender como seria o sinal efetivamente recebido na outra ponta dos nossos canais, devido única e exclusivamente aos efeitos deles sobre o sinal enviado.

Primeiramente, vamos importar as bibliotecas que serão úteis para nós:

In [ ]:
#Importação das bibliotecas
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib as mpl
from scipy import signal
mpl.rcParams['figure.max_open_warning'] = False

## Geração do Sinal Emitido

Agora, vamos gerar um sinal com a forma de uma onda dente-de-serra, mais uma componente constante (DC). Para isso, vamos gerar um vetor t com instantes de tempo linearmente espaçados entre os instantes $t_i$ (instante de tempo inicial da nossa simulação) e $t_f$ (instante de tempo final da nossa simulação). Em seguida, utilizando o módulo signal da biblioteca scipy, vamos gerar o nosso sinal que será aplicado na entrada do canal de comunicações, considerando uma frequência fundamental de $f_0 = 1$ Hz:

In [ ]:
t_i = -3
t_f = 3
passo = 0.00001
t = -np.arange(t_i,t_f,passo)

dc = 0.5

f0 = 1

x = dc + 0.5*signal.sawtooth(2*np.pi*f0*t)

Na célula abaixo, estamos gerando um gráfico do sinal emitido ao longo do tempo:

In [ ]:
fig = plt.figure(1)
ax = fig.add_subplot(1, 1, 1)
plt.plot(t, x,'-b')
plt.title('Sinal Emitido')
plt.xlabel('t (s)')
plt.grid()

## Canais de Comunicações

A seguir, vamos especificar o comportamento dos nossos canais de comunicações considerados nesta simulação. Nela, vamos simular os efeitos um canal passa-baixas de primeira ordem, cujo módulo da resposta em frequência será dado por $$H(f) = \dfrac{1}{\sqrt{1 + \left(\dfrac{f}{f_c}\right)^2}},$$ em que $f_c$ é a frequência de corte do nosso canal. Já a fase da resposta em frequência do nosso canal será dada por $$\varphi(f) = -\arctan\left(\dfrac{f}{f_c}\right).$$ Note que, quanto maior o valor da frequência $f$, menor o valor de $H(f)$, o que significa que frequências muito altas serão fortemente atenuadas, pois suas amplitudes serão multiplicadas por um número próximo de zero. Em contrapartida, para frequências baixas, o valor de $H(f)$ será próximo de 1, o que indica que a amplitude dessas ondas praticamente não será afetada pelo canal.

Para facilitar, vamos implementar o cálculo de $H(f)$ por meio de uma função, como ilustrado abaixo:

In [ ]:
def modulo_resposta_em_frequencia(f,f_c):
    H = 1./np.sqrt(1+(f/f_c)**2)
    return H

Similarmente, podemos implementar o cálculo de $\varphi(f)$ como mostrado a seguir:

In [ ]:
def fase_resposta_em_frequencia(f,f_c):
    Phi = np.arctan(-f/f_c)
    return Phi

Vamos agora gerar os gráficos da resposta em frequência dos canais de comunicações. Para isso, vamos criar um vetor de frequências, calcular o módulo e a fase da resposta em frequência para cada ponto, e plotar os resultados.

In [ ]:
f_final = 50
f = np.arange(0,f_final,0.001)

A resposta em frequência vai variar com o valor da frequência de corte selecionada. Vamos adotar, por exemplo, $f_c = 2$ Hz e analisar os gráficos.

In [ ]:
fc = 2


fig = plt.figure(figsize=(12,10), dpi= 300)
ax1 = fig.add_subplot(2, 1, 1)
#Gráfico do Módulo da Resposta em Frequência:
plt.plot(f, modulo_resposta_em_frequencia(f,fc),'-b')
plt.title('Modulo da Resposta em Frequência')
plt.xlabel(r'$f$')
plt.axis([0, f_final, 0.0, 1.1])
plt.grid()
#Gráfico da Fase da Resposta em Frequência (em graus):
ax2 = fig.add_subplot(2, 1, 2)
plt.plot(f, (180/np.pi)*fase_resposta_em_frequencia(f,fc),'-b')
plt.title('Fase da Resposta em Frequência')
plt.xlabel(r'$f$')
plt.grid()
fig.subplots_adjust(hspace=0.7)

**Note que este NÃO é um canal passa-baixas IDEAL!** O módulo da resposta em frequência não é 1 para frequências menores que 2 Hz, nem 0 para frequências acima desse valor. Lembre-se que, na prática, canais passa-baixa ou passa-faixa *ideais* não existem.

Se aumentarmos o valor da frequência de corte para $f_c = 20$ Hz e plotarmos novamente os gráficos, obteremos:

In [ ]:
fc = 20


fig = plt.figure(figsize=(12,10), dpi= 300)
ax1 = fig.add_subplot(2, 1, 1)
#Gráfico do Módulo da Resposta em Frequência:
plt.plot(f, modulo_resposta_em_frequencia(f,fc),'-b')
plt.title('Modulo da Resposta em Frequência')
plt.xlabel(r'$f$')
plt.axis([0, f_final, 0.0, 1.1])
plt.grid()
#Gráfico da Fase da Resposta em Frequência (em graus):
ax2 = fig.add_subplot(2, 1, 2)
plt.plot(f, (180/np.pi)*fase_resposta_em_frequencia(f,fc),'-b')
plt.title('Fase da Resposta em Frequência')
plt.xlabel(r'$f$')
plt.grid()
fig.subplots_adjust(hspace=0.7)

Comparando os gráficos obtidos com $f_c = 2$ e com $f_c = 20$, vemos que, no segundo caso, a atenuação das frequências mais altas no gráfico é bem menos intensa do que quando tínhamos adotado $f_c = 2$.

### Diante disso, a pergunta que fica é: se tentarmos transmitir o nosso sinal em formato de dente-de-serra plotado anteriormente por esses canais, como ficará o sinal recebido na outra ponta?

Para responder a essa pergunta, vamos recorrer à Análise de Fourier. De posse da resposta em frequência dos canais, conseguimos saber como eles reagem à aplicação de ondas senoidais de diferentes frequências. Precisamos agora entender quais ondas senoidais fazem parte da composição do nosso sinal, e com que amplitudes e fases.

Como o sinal é periódico, sabemos que ele pode ser decomposto por meio da Série de Fourier, em que aparecerão uma componente constante e ondas cujas frequências são múltiplas inteiras da frequência fundamental $f_0$, ou seja, ondas com frequência $f_n = n \cdot f_0$, com $n = 1, 2, 3, \cdots$. Para essa a forma de onda do sinal plotado anteriormente, é possível obter: $$a_0 = \dfrac{1}{2},\quad A_n = \dfrac{1}{\pi n}\quad \text{e}\quad \phi_n = 90^{\circ} \ \text{para qualquer}\ n\ \text{inteiro}.$$ Para verificar isso, vamos comparar o sinal com a sua decomposição até a $n$-ésima harmônica, com $n$ indo de $1$ até $50$ 

In [ ]:
a0 = 0.5

x_fourier = a0*np.ones(len(t))

N = 50

vetor_An = np.zeros(N)
vetor_phi_n = np.zeros(N)

for n in range(1,N+1):
    An = 1/(np.pi*n)
    phi_n = np.pi/2
    vetor_An[n-1] = An
    vetor_phi_n[n-1] = phi_n
    x_fourier = x_fourier + An*np.cos(2*np.pi*n*f0*t + phi_n)
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    plt.plot(t, x,'-b')
    plt.plot(t, x_fourier,'-r')
    plt.title('Sinal Transmitido')
    plt.xlabel('t (s)')
    plt.grid()




Podemos, a partir daí, analisar o espectro do sinal de entrada. Fazendo o gráfico da amplitude e da fase de cada componente do nosso sinal, obtemos:

In [ ]:
f1 = np.arange(f_final+1)

fig = plt.figure(figsize=(6,6), dpi= 300)
ax1 = fig.add_subplot(2, 1, 1)
(markerline, stemlines, baseline) = ax1.stem(f1, np.concatenate((np.asarray([a0]), vetor_An)),'-b', markerfmt=" ", use_line_collection = True)
plt.setp(baseline, visible=False) 
plt.ylabel('Amplitude')
plt.xlabel('f (Hz)')
plt.grid()
ax2 = fig.add_subplot(2, 1, 2)
(markerline, stemlines, baseline) = ax2.stem(f1, (180/np.pi)*np.concatenate((np.asarray([0]), vetor_phi_n)),'-b', markerfmt=" ", use_line_collection = True)
plt.setp(baseline, visible=False) 
plt.ylabel('Fase ($^\circ$)')
plt.xlabel('f (Hz)')
plt.grid()
fig.subplots_adjust(hspace=0.7)

Você deve ter observado que, quanto mais harmônicas somamos, mais a representação do nosso sinal por meio da Série de Fourier se aproxima da forma do sinal original. Agora, podemos estudar o efeito dos canais sobre cada componente. Por exemplo, no gráfico a seguir, estamos plotando o sinal de entrada em azul escuro e, separadamente:

- A componente constante $a_0$, em vermelho
- A onda senoidal com a frequência fundamental $f_0$, ou seja, $A_1 \cos(2\pi f_0t + \phi_1)$, em preto
- A segunda harmônica, $A_2 \cos(4\pi f_0t + \phi_2)$, em magenta
- A terceira harmônica, $A_3 \cos(6\pi f_0t + \phi_3)$, em azul claro

In [ ]:
fig = plt.figure(figsize=(6,4), dpi= 300)
plt.clf()
ax = fig.add_subplot(1, 1, 1)
plot1, = plt.plot(t, x,'-b')
plot2, = plt.plot(t, a0*np.ones(len(t)),'-r')
plot3, = plt.plot(t, vetor_An[0]*np.cos(2*np.pi*1*f0*t + vetor_phi_n[0]),'-k')
plot4, = plt.plot(t, vetor_An[1]*np.cos(2*np.pi*2*f0*t + vetor_phi_n[1]),'-m')
plot5, = plt.plot(t, vetor_An[2]*np.cos(2*np.pi*3*f0*t + vetor_phi_n[2]),'-c')
plt.title('Sinal Transmitido')
plt.legend([plot1,plot2,plot3,plot4,plot5],["Sinal", "Componente DC", "Fundamental", "2ª Harmônica", "3ª Harmônica"], loc = 'upper left')
plt.xlabel('t (s)')
plt.grid()

O que acontece quando tentamos transmitir cada uma dessas componentes pelo canal de comunicação? Neste ponto, é importante lembrar que:

- A amplitude de cada componente senoidal do nosso sinal será **multiplicada** pelo módulo da resposta em frequência do canal
- A fase de cada componente senoidal do nosso sinal será **somada** à fase da resposta em frequência do canal

Vamos começar considerando o canal com frequência de corte $f_c = 2$. Como $H(0) = \dfrac{1}{\sqrt{1 + \left(\dfrac{0}{2}\right)^2}} = 1$, vamos receber na saída $a_0 \cdot H(0) = a_0 \cdot 1= a_0$. Em outras palavras, a componente DC passa pelo nosso canal inalterada:

In [ ]:
fc1 = 2

fig = plt.figure()
plt.clf()
ax1 = fig.add_subplot(1, 2, 1)
plt.plot(t, a0*np.ones(len(t)),'-b')
plt.title('Componente Transmitida')
plt.xlabel('t (s)')
plt.grid()
ax1 = fig.add_subplot(1, 2, 2)
plt.plot(t, a0*modulo_resposta_em_frequencia(0,fc1)*np.ones(len(t)),'-r')
plt.title('Componente Recebida')
plt.xlabel('t (s)')
plt.grid()

Vamos analisar agora o que acontece com a componente com a frequência fundamental $f_0 = 1$, dada por $$A_1\cos(2 \pi t + \phi_1) = \dfrac{1}{\pi}\cos(2 \pi t + 90^{\circ}) \approx 0.318\cos(2 \pi t + 90^{\circ})$$ Neste caso, vamos obter:
$$H(1) = \dfrac{1}{\sqrt{1 + \left(\dfrac{1}{2}\right)^2}} \approx 0.894$$ e $$\varphi = - \arctan\left(\dfrac{1}{2}\right) \approx 0.464 \ \text{radianos ou}\ 26,7^{\circ}$$

Logo, o que vamos observar na saída será uma onda senoidal dada por $$A_1 \cdot H(1)\cdot \cos[2 \pi t + \phi_1 + \varphi(1)] \approx 0.285\cos(2\pi t + 116,6^{\circ})$$


In [ ]:
fc1 = 2

fig = plt.figure()
plt.clf()
ax1 = fig.add_subplot(1, 2, 1)
plt.plot(t, vetor_An[0]*np.cos(2*np.pi*1*f0*t + vetor_phi_n[0]),'-b')
plt.title('Componente Transmitida')
plt.xlabel('t (s)')
plt.axis([t_i, t_f, -0.35, 0.35])
plt.grid()
ax1 = fig.add_subplot(1, 2, 2)
plt.plot(t, vetor_An[0]*modulo_resposta_em_frequencia(f0,fc1)*np.cos(2*np.pi*1*f0*t + vetor_phi_n[0]+ fase_resposta_em_frequencia(f0,fc1)),'-r')
plt.title('Componente Recebida')
plt.axis([t_i, t_f, -0.35, 0.35])
plt.xlabel('t (s)')
plt.grid()

Seguindo a mesma linha de raciocínio, vamos examinar o que acontece com a segunda harmônica, considerando $f_2 = 2$, dada por $$A_2\cos(4 \pi t + \phi_1) = \dfrac{1}{2\pi}\cos(4 \pi t + 90^{\circ}) \approx 0.159\cos(4 \pi t + 90^{\circ})$$ Neste caso, vamos obter:
$$H(2) = \dfrac{1}{\sqrt{1 + \left(\dfrac{2}{2}\right)^2}} \approx 0.707$$ e $$\varphi = - \arctan\left(\dfrac{2}{2}\right) \approx 0.785 \ \text{radianos ou}\ 45^{\circ}$$

Logo, o que vamos observar na saída será uma onda senoidal dada por $$A_2 \cdot H(2)\cdot \cos[4 \pi t + \phi_2 + \varphi(2)] \approx 0.113\cos(2\pi t + 135^{\circ})$$

In [ ]:
fc1 = 2

fig = plt.figure()
plt.clf()
ax1 = fig.add_subplot(1, 2, 1)
plt.plot(t, vetor_An[1]*np.cos(2*np.pi*2*f0*t + vetor_phi_n[1]),'-b')
plt.title('Componente Transmitida')
plt.xlabel('t (s)')
plt.axis([t_i, t_f, -0.35, 0.35])
plt.grid()
ax1 = fig.add_subplot(1, 2, 2)
plt.plot(t, vetor_An[1]*modulo_resposta_em_frequencia(2*f0,fc1)*np.cos(2*np.pi*2*f0*t + vetor_phi_n[1]+ fase_resposta_em_frequencia(2*f0,fc1)),'-r')
plt.title('Componente Recebida')
plt.axis([t_i, t_f, -0.35, 0.35])
plt.xlabel('t (s)')
plt.grid()

Prosseguindo, vamos passar à análise do que ocorre com a terceira harmônica, considerando $f_3 = 3$, dada por $$A_3\cos(6 \pi t + \phi_3) = \dfrac{1}{3\pi}\cos(6 \pi t + 90^{\circ}) \approx 0.106\cos(6 \pi t + 90^{\circ})$$ Neste caso, vamos obter:
$$H(3) = \dfrac{1}{\sqrt{1 + \left(\dfrac{3}{2}\right)^2}} \approx 0.555$$ e $$\varphi = - \arctan\left(\dfrac{2}{2}\right) \approx 0.982 \ \text{radianos ou}\ 56,3^{\circ}$$

Logo, o que vamos observar na saída será uma onda senoidal dada por $$A_3 \cdot H(3)\cdot \cos[6 \pi t + \phi_3 + \varphi(3)] \approx 0.059 \cos(2\pi t + 146,3^{\circ})$$

In [ ]:
fc1 = 2

fig = plt.figure()
plt.clf()
ax1 = fig.add_subplot(1, 2, 1)
plt.plot(t, vetor_An[2]*np.cos(2*np.pi*3*f0*t + vetor_phi_n[2]),'-b')
plt.title('Componente Transmitida')
plt.xlabel('t (s)')
plt.axis([t_i, t_f, -0.35, 0.35])
plt.grid()
ax1 = fig.add_subplot(1, 2, 2)
plt.plot(t, vetor_An[2]*modulo_resposta_em_frequencia(3*f0,fc1)*np.cos(2*np.pi*3*f0*t + vetor_phi_n[2]+ fase_resposta_em_frequencia(3*f0,fc1)),'-r')
plt.title('Componente Recebida')
plt.axis([t_i, t_f, -0.35, 0.35])
plt.xlabel('t (s)')
plt.grid()

Com base nisso, podemos calcular a amplitude e a fase de cada componente do sinal de saída e plotar o seu espectro. Considerando as $N = 50$ primeiras harmônicas, temos:

In [ ]:
# Cálculo da amplitude e da fase de cada harmônica que compõe o sinal de saíad:

N = 50

a0_saida = a0*modulo_resposta_em_frequencia(0,fc1)
vetor_An_sinal_de_saida = np.zeros(N)
vetor_phi_n_sinal_de_saida = np.zeros(N)

for n in range(1,N+1):
    An_entrada = 1/(np.pi*n)
    phi_n_entrada = np.pi/2
    vetor_An_sinal_de_saida[n-1] = An_entrada*modulo_resposta_em_frequencia(n*f0,fc1)
    vetor_phi_n_sinal_de_saida[n-1] = phi_n_entrada + fase_resposta_em_frequencia(n*f0,fc1)

Plotando o resultado:

In [ ]:
f1 = np.arange(f_final+1)

fig = plt.figure(figsize=(6,6), dpi= 300)
ax1 = fig.add_subplot(2, 1, 1)
(markerline, stemlines, baseline) = ax1.stem(f1, np.concatenate((np.asarray([a0_saida]), vetor_An_sinal_de_saida)),'-b', markerfmt=" ", use_line_collection = True)
plt.setp(baseline, visible=False) 
plt.ylabel('Amplitude')
plt.xlabel('f (Hz)')
plt.grid()
ax2 = fig.add_subplot(2, 1, 2)
(markerline, stemlines, baseline) = ax2.stem(f1, (180/np.pi)*np.concatenate((np.asarray([0]), vetor_phi_n_sinal_de_saida)),'-b', markerfmt=" ", use_line_collection = True)
plt.setp(baseline, visible=False) 
plt.ylabel('Fase ($^\circ$)')
plt.xlabel('f (Hz)')
plt.grid()

fig.subplots_adjust(hspace=0.7)
fig.subplots_adjust(wspace=0.7)

Note que o espectro do sinal recebido é **diferente** daquele do sinal emitido, pois o canal está atenuando mais fortemente as componentes com frequência comparativamente mais elevadas. A seguir, os dois espectros são mostrados lado a lado para efeito de comparação.

In [ ]:
f1 = np.arange(f_final+1)

fig = plt.figure(figsize=(6,6), dpi= 300)
ax1 = fig.add_subplot(2, 2, 1)
(markerline, stemlines, baseline) = ax1.stem(f1, np.concatenate((np.asarray([a0]), vetor_An)),'-b', markerfmt=" ", use_line_collection = True)
plt.setp(baseline, visible=False) 
plt.ylabel('Amplitude')
plt.xlabel('f (Hz)')
plt.grid()
plt.title('Sinal Emitido')
ax2 = fig.add_subplot(2, 2, 3)
(markerline, stemlines, baseline) = ax2.stem(f1, (180/np.pi)*np.concatenate((np.asarray([0]), vetor_phi_n)),'-b', markerfmt=" ", use_line_collection = True)
plt.setp(baseline, visible=False) 
plt.ylabel('Fase ($^\circ$)')
plt.xlabel('f (Hz)')
plt.grid()
ax3 = fig.add_subplot(2, 2, 2)
(markerline, stemlines, baseline) = ax3.stem(f1, np.concatenate((np.asarray([a0_saida]), vetor_An_sinal_de_saida)),'-r', markerfmt=" ", use_line_collection = True)
plt.setp(baseline, visible=False) 
plt.ylabel('Amplitude')
plt.xlabel('f (Hz)')
plt.grid()
plt.title('Sinal Recebido')
ax4 = fig.add_subplot(2, 2, 4)
(markerline, stemlines, baseline) = ax4.stem(f1, (180/np.pi)*np.concatenate((np.asarray([0]), vetor_phi_n_sinal_de_saida)),'-r', markerfmt=" ", use_line_collection = True)
plt.setp(baseline, visible=False) 
plt.ylabel('Fase ($^\circ$)')
plt.xlabel('f (Hz)')
plt.grid()

fig.subplots_adjust(hspace=0.4)
fig.subplots_adjust(wspace=0.7)

### Assim como o sinal emitido é o somatório das várias ondas senoidais que encontramos utilizando a Análise de Fourier, o sinao recebido é o somatório das várias ondas senoidais que encontramos agora há pouco, cada uma afetada de uma determinada maneira pelo canal de comunicação###

Por exemplo, se somarmos até a terceira harmônica, obteríamos as seguintes aproximações para o sinal de entrada e para o sinal de saída:

In [ ]:
a0 = 0.5



x_fourier = a0*np.ones(len(t))
y_fourier = a0_saida*np.ones(len(t))



fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plot1,= plt.plot(t, x_fourier,'-b')
plot2,= plt.plot(t, y_fourier,'-r')
plt.title('Entrada e Saída do Canal (Até a 3ª Harmônica)')
plt.legend([plot1,plot2],["Emitido", "Recebido"], loc = 'upper left')
plt.xlabel('t (s)')
plt.grid()

N = 3

for n in range(1,N+1):
    x_fourier = x_fourier + vetor_An[n-1]*np.cos(2*np.pi*n*f0*t + vetor_phi_n[n-1])
    print(vetor_An[n-1])
    y_fourier = y_fourier + vetor_An_sinal_de_saida[n-1]*np.cos(2*np.pi*n*f0*t + vetor_phi_n_sinal_de_saida[n-1])
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    plot1,= plt.plot(t, x_fourier,'-b')
    plot2,= plt.plot(t, y_fourier,'-r')
    plt.title('Entrada e Saída do Canal (Até a 3ª Harmônica)')
    plt.legend([plot1,plot2],["Emitido", "Recebido"], loc = 'upper left')
    plt.xlabel('t (s)')
    plt.grid()

Em geral, para procurar compreender o efeito de um canal, considerar apenas até a terceira harmônica não é suficiente. Vamos reproduzir o cálculo acima, mas agora considerando as 50 primeiras harmônicas:

In [ ]:
a0 = 0.5

fc1 = 2

f0 = 1



y1 = a0_saida*np.ones(len(t))

N = 50

for n in range(1,N+1):
    y1 = y1 + vetor_An_sinal_de_saida[n-1]*np.cos(2*np.pi*n*f0*t + vetor_phi_n_sinal_de_saida[n-1])

    
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plot1, = plt.plot(t, x,'-b')
plot2, = plt.plot(t, y1,'-r')
plt.title('Entrada e Saída do Canal')
plt.legend([plot1,plot2],["Emitido", "Recebido"], loc = 'upper left')
plt.xlabel('t (s)')
plt.grid()

Note que o sinal recebido é claramente distinto do emitido. Como estamos tentando transmitir o sinal azul por um canal **passa-baixas**, as variações mais rápidas do sinal (associada às componentes de alta frequência) tendem a ser **suavizadas**

E se tentássemos transmitir esse mesmo sinal azul, mas agora por um canal passa-baixas com frequência de corte $f_c = 20$?

Neste caso, o efeito do canal sobre as componentes analisadas mais cedo será menos intenso. Por exemplo, com $f_c = 20$, obtemos:

$$H(1) \approx 0,999 \quad \text{e}\quad \varphi(1) \approx 2,9^{\circ}$$

$$H(2) \approx 0,995 \quad \text{e}\quad \varphi(2) \approx 5,7^{\circ}$$

$$H(3) \approx 0,989 \quad \text{e}\quad \varphi(3) \approx 8,5^{\circ}$$


In [ ]:
fc2 = 20

f0 = 1

fig = plt.figure()
ax1 = fig.add_subplot(1, 2, 1)
plt.plot(t, a0*np.ones(len(t)),'-b')
plt.title('Transmitida (DC)')
plt.xlabel('t (s)')
plt.grid()
ax1 = fig.add_subplot(1, 2, 2)
plt.plot(t, a0*modulo_resposta_em_frequencia(0,fc2)*np.ones(len(t)),'-r')
plt.title('Recebida (DC)')
plt.xlabel('t (s)')
plt.grid()

fig = plt.figure()
ax1 = fig.add_subplot(1, 2, 1)
plt.plot(t, vetor_An[0]*np.cos(2*np.pi*1*f0*t + vetor_phi_n[0]),'-b')
plt.title('Transmitida (Fundamental)')
plt.xlabel('t (s)')
plt.axis([t_i, t_f, -0.35, 0.35])
plt.grid()
ax1 = fig.add_subplot(1, 2, 2)
plt.plot(t, vetor_An[0]*modulo_resposta_em_frequencia(f0,fc2)*np.cos(2*np.pi*1*f0*t + vetor_phi_n[0]+ fase_resposta_em_frequencia(f0,fc2)),'-r')
plt.title('Recebida (Fundamental)')
plt.axis([t_i, t_f, -0.35, 0.35])
plt.xlabel('t (s)')
plt.grid()

fig = plt.figure()
ax1 = fig.add_subplot(1, 2, 1)
plt.plot(t, vetor_An[1]*np.cos(2*np.pi*2*f0*t + vetor_phi_n[1]),'-b')
plt.title('Transmitida (2ª Harmônica)')
plt.xlabel('t (s)')
plt.axis([t_i, t_f, -0.35, 0.35])
plt.grid()
ax1 = fig.add_subplot(1, 2, 2)
plt.plot(t, vetor_An[1]*modulo_resposta_em_frequencia(2*f0,fc2)*np.cos(2*np.pi*2*f0*t + vetor_phi_n[1]+ fase_resposta_em_frequencia(2*f0,fc2)),'-r')
plt.title('Recebida (2ª Harmônica)')
plt.axis([t_i, t_f, -0.35, 0.35])
plt.xlabel('t (s)')
plt.grid()

fig = plt.figure()
ax1 = fig.add_subplot(1, 2, 1)
plt.plot(t, vetor_An[2]*np.cos(2*np.pi*3*f0*t + vetor_phi_n[2]),'-b')
plt.title('Transmitida (3ª Harmônica)')
plt.xlabel('t (s)')
plt.axis([t_i, t_f, -0.35, 0.35])
plt.grid()
ax1 = fig.add_subplot(1, 2, 2)
plt.plot(t, vetor_An[2]*modulo_resposta_em_frequencia(3*f0,fc2)*np.cos(2*np.pi*3*f0*t + vetor_phi_n[2]+ fase_resposta_em_frequencia(3*f0,fc2)),'-r')
plt.title('Recebida (3ª Harmônica)')
plt.axis([t_i, t_f, -0.35, 0.35])
plt.xlabel('t (s)')
plt.grid()

Consequentemente, ao transmitir o sinal azul pelo canal com frequência de corte $f_c = 20$, ele chegará muito menos *distorcido* ao receptor:

In [ ]:
a0 = 0.5

fc2 = 20

y2 = a0*modulo_resposta_em_frequencia(0,fc2)*np.ones(len(t))

N = 50

vetor_An = np.zeros(N)
vetor_phi_n = np.zeros(N)

for n in range(1,N+1):
    An = 1/(np.pi*n)
    phi_n = np.pi/2
    vetor_An[n-1] = An
    vetor_phi_n[n-1] = phi_n
    y2 = y2 + An*modulo_resposta_em_frequencia(n*f0,fc2)*np.cos(2*np.pi*n*f0*t + phi_n + fase_resposta_em_frequencia(n*f0,fc2))

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plot1, = plt.plot(t, x,'-b')
plot2, = plt.plot(t, y2,'-r')
plt.title('Entrada e Saída do Canal')
plt.legend([plot1,plot2],["Emitido", "Recebido"], loc = 'upper left')
plt.xlabel('t (s)')
plt.grid()

Por fim, o que aconteceria se tentássemos transmitir agora um sinal com frequência fundamental $f_0 = 100$ por meio do canal com frequência de corte $f_c = 2$?

In [ ]:
a0 = 0.5

f0 = 100

t_i = -0.03
t_f = 0.03

t = -np.arange(t_i,t_f,passo)

x = dc + 0.5*signal.sawtooth(2*np.pi*f0*t)

fc1 = 2

y1 = a0*modulo_resposta_em_frequencia(0,fc1)*np.ones(len(t))

N = 50

vetor_An = np.zeros(N)
vetor_phi_n = np.zeros(N)

for n in range(1,N+1):
    An = 1/(np.pi*n)
    phi_n = np.pi/2
    vetor_An[n-1] = An
    vetor_phi_n[n-1] = phi_n
    y1 = y1 + An*modulo_resposta_em_frequencia(n*f0,fc1)*np.cos(2*np.pi*n*f0*t + phi_n + fase_resposta_em_frequencia(n*f0,fc1))

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plot1, = plt.plot(t, x,'-b')
plot2, = plt.plot(t, y1,'-r')
plt.title('Entrada e Saída do Canal')
plt.legend([plot1,plot2],["Emitido", "Recebido"], loc = 'upper left')
plt.xlabel('t (s)')
plt.grid()

Neste caso, mesmo a componente fundamental é muito atenuada pelo canal. Afinal, com $f_c = 2 Hz$, temos que $H(100) \approx 0,02$. Com isso, o sinal recebido na outra ponta é praticamente dado pela componente cosntante (que não é atenuada pois $H(0) = 1$)